<a href="https://colab.research.google.com/github/Shafiqul-shawon/ComparatorSorting/blob/master/Assignment_5_Data_Preprocessing_and_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
#Load the Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [142]:
# TODO: Load the housing data
df = pd.read_csv('/content/drive/MyDrive/datasets/Housing.csv')

# TODO: How many houses are there?
print(f"Number of houses: {df.shape[0]}")

# TODO: Show first 3 houses
# Your code here

Number of houses: 545


In [143]:
# Look at Columns
# Show all column names
print("Columns:")
print(df.columns.tolist())

# Show data types of columns
print("Data types:")
print(df.dtypes)

Columns:
['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']
Data types:
price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


In [144]:
# Convert Yes/No to 1/0
from sklearn.preprocessing import LabelEncoder

for col in ["mainroad","guestroom","basement",
            "hotwaterheating","airconditioning","prefarea"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Encode furnishingstatus using one-hot encoding
df = pd.get_dummies(df, columns=["furnishingstatus"], drop_first=True, dtype=int)

print("Converted data sample:")
print(df.head())

Converted data sample:
      price  area  bedrooms  bathrooms  stories  mainroad  guestroom  \
0  13300000  7420         4          2        3         1          0   
1  12250000  8960         4          4        4         1          0   
2  12250000  9960         3          2        2         1          0   
3  12215000  7500         4          2        2         1          0   
4  11410000  7420         4          1        2         1          1   

   basement  hotwaterheating  airconditioning  parking  prefarea  \
0         0                0                1        2         1   
1         0                0                1        3         0   
2         1                0                0        2         1   
3         1                0                1        3         1   
4         1                0                1        2         0   

   furnishingstatus_semi-furnished  furnishingstatus_unfurnished  
0                                0                             0  
1

In [145]:
# Split into Train and Test
# TODO: Split - 80% train, 20% test
X = df.drop('price', axis=1)  # Everything except house price
y = df['price']                # Only price

print("X has housing information (Except price) (8 columns)")
print("y has amount of price (1 Column)")

# Split: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 20% for test
    random_state=20   # Same result every time
)

print(f"\nWe will train with: {len(X_train)} Houses")
print(f"We will test with: {len(X_test)} Houses")

X has housing information (Except price) (8 columns)
y has amount of price (1 Column)

We will train with: 436 Houses
We will test with: 109 Houses


In [149]:
#Apply StandardScaler
# Area is huge (1650-13300), Bedrooms is small (1-6)!
#Your Code here
scaler = StandardScaler()

# Learn from training data and scale it
X_train_scaled = scaler.fit_transform(X_train)

# Scale test data (just transform, don't fit)
X_test_scaled = scaler.transform(X_test)


In [150]:
#train and test model
#your code here

from sklearn.linear_model import LogisticRegression

# Create model
model = LogisticRegression(max_iter=10000)

# Train the model
model.fit(X_train_scaled, y_train)
print("✅ Model trained!")

# Check accuracy on training data
train_score = model.score(X_train_scaled, y_train)
print(f"Training accuracy: {train_score:.1%}")

# Check accuracy on test data
test_score = model.score(X_test_scaled, y_test)
print(f"Test accuracy: {test_score:.1%}")

✅ Model trained!
Training accuracy: 45.4%
Test accuracy: 1.8%


In [151]:
import joblib

# Save the model
joblib.dump(model, 'my_model.pkl')
print("✅ Model saved as 'my_model.pkl'")

# Save the scaler
joblib.dump(scaler, 'my_scaler.pkl')
print("✅ Scaler saved as 'my_scaler.pkl'")

# Save scaled data as CSV
pd.DataFrame(X_train_scaled).to_csv('train_data_scaled.csv', index=False)
print("✅ Scaled data saved as 'train_data_scaled.csv'")

✅ Model saved as 'my_model.pkl'
✅ Scaler saved as 'my_scaler.pkl'
✅ Scaled data saved as 'train_data_scaled.csv'


In [153]:
# Predict a House Price
# New house details:
# area=5000, bedrooms=3, bathrooms=2, stories=2,
# mainroad=yes, guestroom=no, basement=yes,
# hotwaterheating=no, airconditioning=yes,
# parking=2, prefarea=yes, furnishingstatus=furnished

model = joblib.load('my_model.pkl')
scaler = joblib.load('my_scaler.pkl')


# Predict a House Price
new_house = pd.DataFrame([{
    "area": 5000, "bedrooms": 3, "bathrooms": 2, "stories": 2,
    "mainroad": 1, "guestroom": 0, "basement": 1,
    "hotwaterheating": 0, "airconditioning": 1,
    "parking": 2, "prefarea": 1,
    "furnishingstatus_semi-furnished": 0,
    "furnishingstatus_unfurnished": 0
}])

new_house_scaled = scaler.transform(new_house)
predicted_price = model.predict(new_house_scaled)[0]
print(f"Predicted price of the new house: {predicted_price:.2f}")


Predicted price of the new house: 6300000.00
